In [2]:
import org.apache.spark.sql.{SQLContext, SaveMode}
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@2802039c


org.apache.spark.sql.SQLContext@2802039c

In [3]:
val PM = spark.read.format("parquet").load("file:///N/dc2/projects/IUNI_MSAcademic/pubMed/2019/*.parquet")
PM.printSchema()

[Stage 0:======================================================>(967 + 5) / 972]root
 |-- Abstract: struct (nullable = true)
 |    |-- AbstractText: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _Label: string (nullable = true)
 |    |    |    |-- _NlmCategory: string (nullable = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- i: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- sup: string (nullable = true)
 |    |-- CopyrightInformation: string (nullable = true)
 |    |-- b: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- i: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- sub: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- sup: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- AbstractText: array (nu

PM = [Abstract: struct<AbstractText: array<struct<_Label:string,_NlmCategory:string,_VALUE:string,i:array<string>,sup:string>>, CopyrightInformation: string ... 4 more fields>, AbstractText: array<struct<_Label:string,_NlmCategory:string,_VALUE:string,b:string,i:array<string>,sub:array<string>,sup:array<string>>> ... 32 more fields]


[Abstract: struct<AbstractText: array<struct<_Label:string,_NlmCategory:string,_VALUE:string,i:array<string>,sup:string>>, CopyrightInformation: string ... 4 more fields>, AbstractText: array<struct<_Label:string,_NlmCategory:string,_VALUE:string,b:string,i:array<string>,sub:array<string>,sup:array<string>>> ... 32 more fields]

In [3]:
sqlContext.sql("set spark.sql.caseSensitive=true") 
val WoS = spark.read.format("parquet").load("/data1/parquet/")
//val WoS = spark.read.format("parquet").load("file:///N/dc2/projects/IUNI_MSAcademic/wos_raw/parquet/*.parquet")
WoS.printSchema()

root
 |-- UID: string (nullable = true)
 |-- _r_id_disclaimer: string (nullable = true)
 |-- abstracts: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- abstract: struct (nullable = true)
 |    |    |-- abstract_text: struct (nullable = true)
 |    |    |    |-- _count: long (nullable = true)
 |    |    |    |-- p: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |-- addresses: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- address_name: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- address_spec: struct (nullable = true)
 |    |    |    |    |-- _addr_no: long (nullable = true)
 |    |    |    |    |-- city: string (nullable = true)
 |    |    |    |    |-- country: string (nullable = true)
 |    |    |    |    |-- full_address: string (nullable = true)
 |    |    |    |    |-- organizations: struct 

WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]


[UID: string, _r_id_disclaimer: string ... 13 more fields]

In [50]:
val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.abstracts.abstract.abstract_text.p","static_data.summary.doctypes.doctype",
                       "static_data.summary.pub_info._pubyear","static_data.summary.names.name.display_name","static_data.summary.publishers.publisher.names.name.full_name","static_data.summary.titles.title")
WoS2.printSchema
val PM2 = PM.select("MedlineCitation.PMID._VALUE","PubmedData.ArticleIdList.ArticleId","MedlineCitation.Abstract.AbstractText._VALUE","MedlineCitation.OtherID","MedlineCitation.Article.PublicationTypeList.PublicationType",
                    "MedlineCitation.DateCompleted","MedlineCitation.Article.AuthorList.Author","KeywordList.Keyword","MedlineCitation.Article.ArticleTitle")
                    .toDF("PMID","ArticleId","Abstract","OtherID","PubType","Date","Author","Keyword","Title")
PM2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- p: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- _pubyear: long (nullable = true)
 |-- display_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- full_name: string (nullable = true)
 |-- title: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _translated: string (nullable = true)
 |    |    |-- _type: string (nullable = true)

root
 |-- PMID: long (nullable = true)
 |-- ArticleId: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _IdType: string (

WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 6 more fields]
PM2 = [PMID: bigint, ArticleId: array<struct<_IdType:string,_VALUE:string>> ... 7 more fields]


lastException: Throwable = null


[PMID: bigint, ArticleId: array<struct<_IdType:string,_VALUE:string>> ... 7 more fields]

In [5]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.abstracts.abstract.abstract_text.p","static_data.summary.doctypes.doctype",
                       "static_data.summary.pub_info._pubyear","static_data.summary.names.name.display_name","static_data.summary.publishers.publisher.names.name.full_name","static_data.summary.titles.title")

val WoS3 = WoS2.withColumn("identifiers", explode(col("identifier")))
WoS3.groupBy("identifiers._type").count().show

+---------------+-------+
|          _type|  count|
+---------------+-------+
|            doi|2434098|
|           isbn| 640776|
|         art_no| 341849|
|    meeting_abs| 291121|
|          eissn|1413610|
|          eisbn| 342701|
|   accession_no|8466483|
|           issn|7848634|
|parent_book_doi|  62926|
+---------------+-------+



WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 6 more fields]
WoS3 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 7 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 7 more fields]

In [8]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct

def getID(_type: String) = {
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, s, _) => s == _type}
      .map(_._3)
      .orElse(None)
  )
}


val WoS4 = WoS2.withColumn("doi", when(col("identifier").isNotNull, getID("doi")($"identifier")).otherwise(lit(null)))
.select("UID","doi","_pubyear").filter($"doi" =!= "null").coalesce(500)

WoS4.describe().filter($"summary" === "count").show

+-------+-------+-------+--------+
|summary|    UID|    doi|_pubyear|
+-------+-------+-------+--------+
|  count|2434097|2434097| 2434097|
+-------+-------+-------+--------+



WoS4 = [UID: string, doi: string ... 1 more field]


getID: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, doi: string ... 1 more field]

In [13]:
WoS4.show

+-------------------+--------------------+
|                UID|                 doi|
+-------------------+--------------------+
|WOS:000314473000028|  10.1039/c2an36575h|
|WOS:000215261800133| 10.1155/2013/524603|
|WOS:000318694100266|                null|
|WOS:000318694100428|                null|
|WOS:000330468400149|                null|
|WOS:000327808700032|10.1016/j.jvir.20...|
|WOS:000322828100039|10.1128/AEM.01430-13|
|WOS:000312965100005|10.1053/j.gastro....|
|WOS:000332833800288|                null|
|WOS:000316290900022|10.1007/s11669-01...|
|WOS:000330358200026|10.1017/S00098388...|
|WOS:000331980300050|                null|
|WOS:000331980300212|                null|
|WOS:000314627700035|   10.1021/ic3021904|
|WOS:000328683600017| 10.1189/jlb.0812431|
|WOS:000318665300019|10.1016/j.microc....|
|WOS:000339363800092|10.4028/www.scien...|
|WOS:000339363800254|10.4028/www.scien...|
|WOS:000339363800416|10.4028/www.scien...|
|WOS:000218628800005|10.9770/jesi.2013...|
+----------

In [8]:
val PM3 = PM2.withColumn("ArticleIds", explode(col("ArticleId")))
PM3.groupBy("ArticleIds._IdType").count().show

[Stage 8:======================================================>(293 + 1) / 294]+-------+--------+
|_IdType|   count|
+-------+--------+
|    doi|12834762|
|    pmc| 4488017|
|    mid|  659094|
|  pmcid|     262|
|  pmpid|       3|
|medline|      15|
|    pii|10205708|
| pubmed|28577636|
+-------+--------+



PM3 = [PMID: bigint, ArticleId: array<struct<_IdType:string,_VALUE:string>> ... 8 more fields]


[PMID: bigint, ArticleId: array<struct<_IdType:string,_VALUE:string>> ... 8 more fields]

In [31]:
val PM33 = PM3.withColumn("OtherIDs", explode(col("OtherID")))
PM33.groupBy("OtherIDs._Source").count().show

[Stage 124:====================================================>(293 + 1) / 294]+-------+-------+
|_Source|  count|
+-------+-------+
|    CPC|   2668|
|    POP|  81116|
|   CLML|1403484|
|  NRCBL|  24684|
|   ARPL|   1566|
|   NASA|  89262|
|    PIP|  63975|
|    KIE| 111907|
|   CPFH|   2529|
|    NLM|  97867|
|    IND|  16258|
+-------+-------+



PM33 = [PMID: bigint, ArticleId: array<struct<_IdType:string,_VALUE:string>> ... 9 more fields]


[PMID: bigint, ArticleId: array<struct<_IdType:string,_VALUE:string>> ... 9 more fields]

In [9]:
def getIDpm(_type: String) = {
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1)))
      .find { case (s, _) => s == _type}
      .map(_._2)
      .orElse(None)
  )
}

val PM4 = PM2.withColumn("doi", when(col("ArticleId").isNotNull, getIDpm("doi")($"ArticleId")).otherwise(lit(null)))
PM4.describe().filter($"summary" === "count").show

Name: Compile Error
Message: <console>:68: error: not found: value PM2
       val PM4 = PM2.withColumn("doi", when(col("ArticleId").isNotNull, getIDpm("doi")($"ArticleId")).otherwise(lit(null)))
                 ^

StackTrace: 

In [12]:
val PM4 = PM3.filter($"ArticleIds._IdType" === "doi").select("PMID","ArticleIds._VALUE")
PM4.describe().filter($"summary" === "count").show

[Stage 57:=====================================================(298 + -4) / 294]+-------+--------+--------+
|summary|    PMID|  _VALUE|
+-------+--------+--------+
|  count|12834762|12834742|
+-------+--------+--------+



PM4 = [PMID: bigint, _VALUE: string]


lastException: Throwable = null


[PMID: bigint, _VALUE: string]

In [54]:
val PMwos = PM4.join(WoS4, PM4("_VALUE")===WoS4("doi"), "inner")
PMwos.describe().filter($"summary" === "count").show

[Stage 109:====================================================>(198 + 2) / 200]+-------+-------+-------+-------+-------+--------+
|summary|   PMID| _VALUE|    UID|    doi|_pubyear|
+-------+-------+-------+-------+-------+--------+
|  count|8392844|8392844|8392844|8392844| 8392844|
+-------+-------+-------+-------+-------+--------+



PMwos = [PMID: bigint, _VALUE: string ... 3 more fields]


[PMID: bigint, _VALUE: string ... 3 more fields]

In [29]:
val WoS5 = WoS.select("UID","static_data.citedWork","static_data.reference","static_data.fullrecord_metadata.citedWork","static_data.fullrecord_metadata.reference","static_data.fullrecord_metadata.references")
            .toDF("UID","1","2","3","4","5")
//WoS5.describe().filter($"summary" === "count").show

WoS5 = [UID: string, 1: string ... 4 more fields]


[UID: string, 1: string ... 4 more fields]

In [31]:
WoS5.printSchema

root
 |-- UID: string (nullable = true)
 |-- 1: string (nullable = true)
 |-- 2: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- art_no: string (nullable = true)
 |    |    |-- citedAuthor: string (nullable = true)
 |    |    |-- citedTitle: string (nullable = true)
 |    |    |-- citedWork: string (nullable = true)
 |    |    |-- doi: string (nullable = true)
 |    |    |-- i: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- page: string (nullable = true)
 |    |    |-- patent_no: string (nullable = true)
 |    |    |-- sub: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sup: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- uid: string (nullable = true)
 |    |    |-- volume: string (nullable = true)
 |    |    |-- year: string (nullable = true)
 |-- 3: string (nullable = true)
 |-- 4: array (nullable =

lastException: Throwable = null


In [35]:
WoS5.select("UID","1", "3", "2.uid","4.uid","5.reference.uid").show(false)

+-------------------+----+----+----+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

lastException: Throwable = null


In [57]:
val WoSref = WoS5.select("UID","5.reference.uid").toDF("citing","citedList")
val WoSref2 = WoSref.join(PMwos, WoSref("citing")===PMwos("UID"), "inner")
WoSref2.describe().filter($"summary" === "count").show

[Stage 124:================================================>   (188 + 12) / 200]+-------+-------+-------+-------+-------+-------+--------+
|summary| citing|   PMID| _VALUE|    UID|    doi|_pubyear|
+-------+-------+-------+-------+-------+-------+--------+
|  count|8392844|8392844|8392844|8392844|8392844| 8392844|
+-------+-------+-------+-------+-------+-------+--------+



WoSref = [citing: string, citedList: array<string>]
WoSref2 = [citing: string, citedList: array<string> ... 5 more fields]


lastException: Throwable = null


[citing: string, citedList: array<string> ... 5 more fields]

In [60]:
val WoSref3 = WoSref2.withColumn("cited", explode(col("citedList"))).select("citing","PMID","doi","_pubyear","cited")
                .toDF("citingWoS","citingPMID","citingDOI","citingYear","cited")
WoSref3.describe().filter($"summary" === "count").show

[Stage 137:====================================================>(198 + 2) / 200]+-------+---------+----------+---------+----------+---------+
|summary|citingWoS|citingPMID|citingDOI|citingYear|    cited|
+-------+---------+----------+---------+----------+---------+
|  count|321914005| 321914005|321914005| 321914005|321856561|
+-------+---------+----------+---------+----------+---------+



WoSref3 = [citingWoS: string, citingPMID: bigint ... 3 more fields]


lastException: Throwable = null


[citingWoS: string, citingPMID: bigint ... 3 more fields]

In [66]:
val PMwos2 = PMwos.select("UID","PMID","doi","_pubyear").toDF("citedWoS","citedPMID","citedDOI","citedYear")
PMwos2.describe().filter($"summary" === "count").show

[Stage 169:=============================================>           (4 + 1) / 5]+-------+--------+---------+--------+---------+
|summary|citedWoS|citedPMID|citedDOI|citedYear|
+-------+--------+---------+--------+---------+
|  count| 8392844|  8392844| 8392844|  8392844|
+-------+--------+---------+--------+---------+



PMwos2 = [citedWoS: string, citedPMID: bigint ... 2 more fields]


lastException: Throwable = null


[citedWoS: string, citedPMID: bigint ... 2 more fields]

In [69]:
val WoSref4 = WoSref3.join(broadcast(PMwos2), WoSref3("cited")===PMwos2("citedWoS"), "inner")
WoSref4.describe().filter($"summary" === "count").show

[Stage 199:====================================================>(129 + 1) / 130]+-------+---------+----------+---------+----------+---------+---------+---------+---------+---------+
|summary|citingWoS|citingPMID|citingDOI|citingYear|    cited| citedWoS|citedPMID| citedDOI|citedYear|
+-------+---------+----------+---------+----------+---------+---------+---------+---------+---------+
|  count|118949320| 118949320|118949320| 118949320|118949320|118949320|118949320|118949320|118949320|
+-------+---------+----------+---------+----------+---------+---------+---------+---------+---------+



WoSref4 = [citingWoS: string, citingPMID: bigint ... 7 more fields]


lastException: Throwable = null


[citingWoS: string, citingPMID: bigint ... 7 more fields]

In [74]:
WoSref4.coalesce(1).write.mode(SaveMode.Overwrite).format("com.databricks.spark.csv")
    .option("header", "true")
    .option("codec", "org.apache.hadoop.io.compress.GzipCodec")
    .save("file:///N/dc2/projects/IUNI_MSAcademic/pubMed/WoSmatch.csv.gz")

[Stage 211:======================================================>(69 + 1) / 70]

In [5]:
WoSref4.write.mode(SaveMode.Overwrite).parquet("file:///N/dc2/projects/IUNI_MSAcademic/pubMed/WoSmatch/")

Name: Unknown Error
Message: <console>:38: error: not found: value WoSref4
       WoSref4.write.mode(SaveMode.Overwrite).parquet("file:///N/dc2/projects/IUNI_MSAcademic/pubMed/WoSmatch/")
       ^

StackTrace: 

In [2]:
val WoSref5 = spark.read.format("csv").option("header", "true").load("file:///N/dc2/projects/IUNI_MSAcademic/pubMed/WoSmatch.csv.gz")
WoSref5.describe().filter($"summary" === "count").show

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Master removed our application: KILLED
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1487)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1486)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1486)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:814)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:814)
  at scala.Option.foreach(Option.scala:257)
  at org.apache.spark.scheduler.DAGSched

In [9]:
WoSref5.repartition(50).write.mode(SaveMode.Overwrite).parquet("file:///N/dc2/projects/IUNI_MSAcademic/pubMed/WoSmatch/")

[Stage 13:======================================================> (49 + 1) / 50]